# 2D Ising Model - Fast

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import ProgressBar

In [ ]:
%load_ext cython

In [ ]:
# slow python version
def py_metropolis_step(state, kT, j, b, nSteps=1):
    for _ in range(nSteps):
        N, M = state.shape #len(state), len(state[0])
        idx, idy = np.random.randint(N), np.random.randint(M)
        deltaE = 2*state[idx,idy]*b + 2*j*state[idx,idy]*(state[(idx-1) % N,idy] + state[(idx+1) % N,idy]
                                                     +state[idx,(idy-1) % M] + state[idx,(idy+1)%M])
        if deltaE < 0 or np.random.uniform() < np.exp(-deltaE/kT):
            state[idx,idy] *= -1

In [ ]:
%%cython -a
# cython: boundscheck=False, initializedcheck=False, overflowcheck=False, cdivision=True
# xcython:  profile=True, binding=True
# xcython: linetrace=True
# xdistutils: define_macros=CYTHON_TRACE_NOGIL=1
# distutils: extra_compile_args=["-O3", "-fopenmp"]
# distutils: extra_link_args="-fopenmp"

cimport cython as c
cimport numpy as np
import numpy as np

from libc.stdlib cimport rand
from libc.math cimport exp

cdef extern from "limits.h":
    int RAND_MAX

def random_state(m: c.int, n: c.int):
    cdef int[:,::1] state = np.random.randint(0,2,(m,n)).astype(np.intc)*2 - 1
    return state

def metropolis_step(state: c.int[:,::1], kT: c.double, j: c.double, b: c.double, nSteps: c.int = 1) -> None:
# cdef void metropolis_step(int[:,::1] state, double kT, double j, double b, int nSteps):
    cdef int i
    cdef int N, M
    N = state.shape[0]
    M = state.shape[1]
    cdef double deltaE
    cdef int idx, idy
    for i in range(nSteps):
        idx, idy = rand() % N, rand() % M
        deltaE = 2*state[idx,idy]*b + 2*j*state[idx,idy]*(state[(idx-1) % N,idy] + state[(idx+1) % N,idy]
                                                     +state[idx,(idy-1) % M] + state[idx,(idy+1)%M])
        if deltaE < 0 or <double>rand()/RAND_MAX < exp(-deltaE/kT):
            state[idx,idy] *= -1

In [ ]:
s = random_state(100,100)

In [ ]:
metropolis_step(s,2.27,1,0,100000)

In [ ]:
plt.matshow(s)

In [ ]:
%timeit metropolis_step(s, 3, 1, 0, 1000)

In [ ]:
%timeit py_metropolis_step(s, 3, 1, 0, 1000)

In [ ]:
def energy(state, j, b):
    return -b*np.sum(state) -j*np.sum(state*(np.roll(state,1,axis=0) + np.roll(state,1,axis=1)))

In [ ]:
def avg_mag(kT, j, b, N=100, Nsamples=1000, chunk_size=100):
    mags = np.zeros(Nsamples)
    ens = np.zeros(Nsamples)
    for i in range(Nsamples):
        if i % chunk_size == 0:
            s = random_state(N,N)
            metropolis_step(s, kT, j, j, N**2)
            metropolis_step(s, kT, j, b, 10*N**2) # anneal
        metropolis_step(s, kT, j, b, N**2) # burn N^2 steps to get an indep. sample
        mags[i] = np.sum(s)
        ens[i] = energy(s, j, b)
    return np.mean(mags), np.mean(ens)

In [ ]:
temps = np.linspace(0.1,4,25)
mags = np.zeros_like(temps)
ens = np.zeros_like(temps)
progress = ProgressBar(len(temps))
for i in progress:
    mags[i], ens[i] = avg_mag(temps[i],1,0,N=30,Nsamples=10000,chunk_size=300)

In [ ]:
# make smooth curves using a rolling average of Navg temperature values
Navg = 2 # must be even
cmags = np.cumsum(mags)
smags = (cmags[Navg:] - cmags[:-Navg])/Navg
cens = np.cumsum(ens)
sens = (cens[Navg:] - cens[:-Navg])/Navg
stemps = temps[Navg//2:-Navg//2]

In [ ]:
plt.plot(temps,mags), plt.plot(stemps, smags)

In [ ]:
plt.plot(stemps, np.gradient(sens))
plt.plot(stemps, -np.gradient(smags))

Try to find critical temperature

In [ ]:
stemps[np.argmin(np.gradient(smags))]

In [ ]:
stemps[np.argmax(np.gradient(sens))]

In [ ]:
s = random_state(50,50)
metropolis_step(s,1,1,0.01,200000)
plt.matshow(s)